In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
df=pd.read_csv('train_hr.csv')
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [4]:
def clean(df):
    df.drop(['employee_id','education','no_of_trainings','previous_year_rating'],axis=1,inplace=True)
    df.gender.replace({'m':1,'f':0},inplace=True)
    df.region=df.region.apply(lambda x:x.split('_')[1])
    df['Age']=pd.cut(df.age,[19,25,40,50,60],labels=[1,2,3,4])
    df['length_of_service']=pd.cut(df.length_of_service,[0,10,20,40],labels=[1,2,3])
    dep=pd.get_dummies(df.department,drop_first=True)
    rec=pd.get_dummies(df.recruitment_channel,drop_first=True)
    df=pd.concat([df,dep,rec],axis=1)
    df.drop(['age','department','recruitment_channel'],axis=1,inplace=True)
    return df

In [5]:
df=clean(df)
df.head()

,region,gender,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age,Finance,HR,Legal,Operations,Procurement,R&D,Sales & Marketing,Technology,referred,sourcing
0,7,0,1,1,0,49,0,2,0,0,0,0,0,0,1,0,0,1
1,22,1,1,0,0,60,0,2,0,0,0,1,0,0,0,0,0,0
2,19,1,1,0,0,50,0,2,0,0,0,0,0,0,1,0,0,1
3,23,1,1,0,0,50,0,2,0,0,0,0,0,0,1,0,0,0
4,26,1,1,0,0,73,0,3,0,0,0,0,0,0,0,1,0,0


In [6]:
df.isnull().sum()

region                0
gender                0
length_of_service     0
KPIs_met >80%         0
awards_won?           0
avg_training_score    0
is_promoted           0
Age                   0
Finance               0
HR                    0
Legal                 0
Operations            0
Procurement           0
R&D                   0
Sales & Marketing     0
Technology            0
referred              0
sourcing              0
dtype: int64

In [7]:
x=df.drop('is_promoted',axis=1)
y=df['is_promoted']

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
scoring='accuracy'
kf=KFold(n_splits=10,random_state=1,shuffle=True)

In [9]:
from sklearn.tree import DecisionTreeClassifier
list=[10,50,100,500]
for i in list:
    des=DecisionTreeClassifier(max_leaf_nodes=i,random_state=1)
    score=cross_val_score(des,x,y,cv=kf,scoring=scoring).mean()
    print("..",i)
    print(score)

.. 10
0.9283498102935557
.. 50
0.9396620375364397
.. 100
0.9406107795077088
.. 500
0.9376732494603122


In [10]:
from sklearn.tree import DecisionTreeClassifier
list=[95,100,110]
for i in list:
    des=DecisionTreeClassifier(max_leaf_nodes=i,random_state=1)
    score=cross_val_score(des,x,y,cv=kf,scoring=scoring).mean()
    print("..",i)
    print(score)

.. 95
0.9405560449702156
.. 100
0.9406107795077088
.. 110
0.9405742898160467


In [11]:
des=DecisionTreeClassifier(max_leaf_nodes=100,random_state=1)
score=cross_val_score(des,x,y,cv=kf,scoring=scoring).mean()
score

0.9406107795077088

# descision tree -- 94.06

In [12]:
from sklearn.ensemble import RandomForestClassifier
list=[500,1000,1500,2000]
for i in list:
    rf=RandomForestClassifier(max_leaf_nodes=i,random_state=1)
    score=cross_val_score(rf,x,y,cv=kf,scoring=scoring).mean()
    print("..",i)
    print(score)

.. 500
0.9360312066768145
.. 1000
0.9385673001756567
.. 1500
0.9391876482393723
.. 2000
0.9381841351077445


In [13]:
rf=RandomForestClassifier(max_leaf_nodes=1500,random_state=1)
score=cross_val_score(rf,x,y,cv=kf,scoring=scoring).mean()
score

0.9391876482393723

# random forest -- 93.92

In [14]:
from sklearn.linear_model import LogisticRegression
lreg=LogisticRegression()
score=cross_val_score(lreg,x,y,cv=kf,scoring=scoring).mean()
score

0.9214895285238855

# logistic reg -- 92.15

In [15]:
from sklearn.svm import SVC
svc=SVC()
score=cross_val_score(svc,x,y,cv=kf,scoring=scoring).mean()
score

0.9148298734713283

# svc -- 91.48

In [16]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
score=cross_val_score(nb,x,y,cv=kf,scoring=scoring).mean()
score

0.8991387367375285

# naive bayes -- 89.91

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
score=cross_val_score(knn,x,y,cv=kf,scoring=scoring).mean()
score

0.9256860062032475

In [18]:
accuracy_rate=[]
for i in range(1,40):
    knn=KNeighborsClassifier(n_neighbors=i)
    score=cross_val_score(knn,x,y,cv=kf,scoring=scoring).mean()
    accuracy_rate.append(score)
np.max(accuracy_rate)

0.9277477203930766

# knn -- 92.77 

# best model------ Descision tree

In [19]:
df.to_csv('train_modified.csv',index=False)

In [20]:
df=pd.read_csv('test_hr.csv')
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [21]:
df=clean(df)
train=pd.read_csv('train_modified.csv')
train.head()

,region,gender,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age,Finance,HR,Legal,Operations,Procurement,R&D,Sales & Marketing,Technology,referred,sourcing
0,7,0,1,1,0,49,0,2,0,0,0,0,0,0,1,0,0,1
1,22,1,1,0,0,60,0,2,0,0,0,1,0,0,0,0,0,0
2,19,1,1,0,0,50,0,2,0,0,0,0,0,0,1,0,0,1
3,23,1,1,0,0,50,0,2,0,0,0,0,0,0,1,0,0,0
4,26,1,1,0,0,73,0,3,0,0,0,0,0,0,0,1,0,0


In [22]:
des=DecisionTreeClassifier(max_leaf_nodes=100,random_state=1)
des.fit(x,y)
df['is_promoted']=des.predict(df)

In [23]:
submit=pd.read_csv('sample_submission_hr.csv')
submit.head()

,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [24]:
submit.is_promoted=df.is_promoted
submit.to_csv('final_submit.csv',index=False)